In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTrajectories
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.ode_agent_jax import ODEAgent_jax
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import torch.nn as nn
import torch
import gym
from cs285.infrastructure import pytorch_util as ptu
from torchdiffeq import odeint
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax

In [7]:
key = jax.random.PRNGKey(0)
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)
device = "gpu" if torch.cuda.is_available() else "cpu"

cpu


In [8]:
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
mb_agent_jax = ODEAgent_jax(
    env=env,
    hidden_size=128,
    num_layers=4,
    ensemble_size=10,
    train_timestep=0.005,
    mpc_horizon_steps=100,
    mpc_timestep=0.005,
    mpc_strategy="random",
    mpc_num_action_sequences=10,
    key=key
)
replay_buffer = ReplayBufferTrajectories(seed=42)
trajs, _ = sample_n_trajectories(env, RandomPolicy(env=env), ntraj=10, max_length=200)
replay_buffer.add_rollouts(trajs)

100%|██████████| 10/10 [00:00<00:00, 45.69it/s]


In [ ]:
for n in trange(2):
    for i in range(mb_agent_jax.ensemble_size):
        traj = replay_buffer.sample_rollout()
        mb_agent_jax.update(i, traj["observations"], traj["actions"], jnp.cumsum(traj["dts"]))
# TODO: change the implementation to allow sampling batched rollouts